In [1]:
# Для формировнаия PDF
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.colors import Color
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

from time import time

# Для расчета средних
import statistics

# Для чтения и записи PDF, оптического анализа текста и перевода PDF -> PNG
from PyPDF2 import PdfFileReader, PdfFileWriter
import cv2
import pytesseract
import fitz

# Для сжатия PDF
from PDFNetPython3.PDFNetPython import PDFDoc, Optimizer, SDFDoc, PDFNet


def compress_pdf(pdf_path):
    # Сжимаем полученный PDF
    # Initialize the library
    PDFNet.Initialize()
    doc = PDFDoc(pdf_path)
    # Optimize PDF with the default settings
    doc.InitSecurityHandler()
    # Reduce PDF size by removing redundant information and compressing data streams
    Optimizer.Optimize(doc)
    doc.Save(pdf_path, SDFDoc.e_linearized)
    doc.Close()


def scan_num_pages_txt(pdf_path, num_pages, output_path, zoom):
    # Текстовый файл, в который записываем текст документа
    f = open(output_path, "w", encoding = 'utf8')
    
    # Файл, который будем распознавать
    doc = fitz.open(pdf_path)

    for num_page in range(num_pages):
        try:
            img = get_png_from_pdf(doc, num_page, zoom)

            # Распознавание текста
            config = r'--oem 3 --psm 6'
            text = pytesseract.image_to_string(img, config = config, lang = 'rus+eng')
            # Убираем переносы строк
            text = text.replace('-\n', '')
            # Записываем в текстовый файл
            f.write(text)

            print("Распозналась страница №{}".format(num_page + 1))
        except Exception as e:
            print(e)
    
    f.close()


def create_num_pages_pdf(pdf_path, num_pages, output_path, zoom, transparency, flag_all = 'default'):
    # Файл, который будем распознавать
    doc = fitz.open(pdf_path)

    # Получаем размер нашего PDF файла
    input_file = PdfFileReader(pdf_path)
    page_size = (input_file.getPage(0).mediaBox[2], input_file.getPage(0).mediaBox[3])
    if flag_all == 'all':
        num_pages = input_file.numPages

    # Создаем новый PDF файл - результат нашей программы
    output_file = PdfFileWriter()

    for num_page in range(num_pages):
        try:    
            img = get_png_from_pdf(doc, num_page, zoom)

            # Данные о координатах элементов, их высоте и т. п.
            config = r'--oem 3 --psm 6'
            data = pytesseract.image_to_data(img, config = config, lang = 'rus+eng')

            path_watermark = "watermark.pdf"
            canvas = create_watermark(path_watermark, page_size, transparency)

            # Длина PNG файла
            y_len_png = img.shape[0]

            fill_watermark(canvas, data, y_len_png)

            # Выводим распозанный текст в PDF
            watermark = PdfFileReader(path_watermark)
            input_page = watermark.getPage(0)
            if transparency:
                input_page.mergePage(input_file.getPage(num_page))
            output_file.addPage(input_page)

            print("Распозналась страница №{}".format(num_page + 1))
        except Exception as e:
            print(e)

    # Сохраняем наш PDF с результатом
    with open(output_path, "wb") as outputStream:
        output_file.write(outputStream) 


def fill_watermark(canvas, data, y_len_png):
    h_list, x_list, words, y_line = [], [], [], 0

    for i, el in enumerate(data.splitlines()):
        if i == 0:
            continue

        el = el.split()
        try:
            # Собираем данные о координатах, ширине и высоте
            x, y, w, h = int(el[6]), int(el[7]), int(el[8]), int(el[9])
            word = el[11]
            # Записываем координату y первого слова
            if not words:
                y_line = y
            x_list.append(x)
            words.append(word)
            # Формирумем список вы
            h_list.append(h)
        except IndexError:
            # Сюда заходит в случае переноса строки
            write_line_to_watermark(canvas, words, x_list, h_list, y_line, y_len_png, zoom)

            # Обнуляем данные по строке
            h_list, x_list, words, y_line = [], [], [], 0

    # Если последняя строка не записалась    
    write_line_to_watermark(canvas, words, x_list, h_list, y_line, y_len_png, zoom)
    
    canvas.save()

def write_line_to_watermark(canvas, words, x_list, h_list, y_line, y_len_png, zoom):
    if words:
        # Берем шрифт примерно средний по строке
        h_avg = (statistics.mean(h_list) + statistics.median(h_list)) / 2
        canvas.setFont('DejaVuSerif', h_avg / zoom)
    for num_word in range(len(words)):
        canvas.drawString(x_list[num_word] / zoom, (y_len_png - y_line - 0.75 * h_avg) / zoom, words[num_word])


def create_watermark(path_watermark, page_size, transparency):
    # Создаем прозрачный pdf с текстом страницы
    canvas = Canvas(path_watermark, pagesize = page_size)
    # Создаем прозрачный цвет
    if transparency == 1:
        transparent_color = Color(0, 0, 0, alpha = 0)
        canvas.setFillColor(transparent_color)
    return canvas


def get_png_from_pdf(pdf_doc, num_page, zoom):
    page = pdf_doc.load_page(num_page)
    # Качество распознавани
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix = mat)
    # Сохраняем страницу в формате PNG
    output = "outfile.png"
    pix.save(output)
    # Считываем страницу для распознавания
    return cv2.imread(output)


# Создаем шрифт, поддерживающий кириллицу
pdfmetrics.registerFont(TTFont('DejaVuSerif','DejaVuSerif.ttf', 'UTF-8'))

pdf_path = "xxx.pdf"
output_path = 'yyy.pdf'
num_pages = 1
zoom = 3
transparency = 1
flag_all = 'all'

create_num_pages_pdf(pdf_path, num_pages, output_path, zoom, transparency, flag_all)
compress_pdf(output_path)

output_txt = 'zzz.txt'
scan_num_pages_txt(pdf_path, num_pages, output_txt, zoom)

Распозналась страница №1
Распозналась страница №1


In [24]:
from PDFNetPython3.PDFNetPython import PDFDoc, Optimizer, SDFDoc, PDFNet